# Using native Python

## Default local client

When rgxlog is imported, a default client is created behind the scenes. This is the client that %%spanner uses.
However, one can also use his own client in the following manner:

In [ ]:
from rgxlog import Client

client = Client()
result = client.execute('''
    new uncle(str, str)
    uncle("bob", "greg")
    ?uncle(X,Y)
    ''')
print(result)
# Notice the default port is taken by the default client, hence the next port is used for the user's client

In [ ]:
client = None  # Notice the connection closes, due to GC
# client.disconnect()  # if you wish to disconnect explicitly you can do so as well

## Manually configured remote client

The following cell demonstrates connecting to a remote server.
* Start the remote server (run server.py)
* Note down the server's port and ip
* Use the following code snippet with your port and ip to connect

In [ ]:
from rgxlog import Client
# the server can also be remote

custom_client = Client(
    remote_ip='192.168.0.100',
    remote_port=32678
)

result = custom_client.execute('''
    new uncle(str, str)
    uncle("bob", "greg")
    ?uncle(X,Y)
    ''')
print(result)

custom_client.disconnect()

# Mixing magics and native python

In [ ]:
import rgxlog  # default client starts here, and captures 32768
from rgxlog import Client

another_client=Client()  # therefore this client will try the next in line, 32769

In [ ]:
%%spanner
# still using the default client

new uncle(str, str)
uncle("bob", "greg")
?uncle(X,Y)

In [ ]:
default_client = rgxlog.magic_client  # save the default for later (prevents GC)
rgxlog.magic_client = another_client

In [ ]:
%%spanner
# using 'another_client', not the default

new uncle(str, str)
uncle("bob", "greg")
?uncle(X,Y)

In [ ]:
another_client.disconnect()
rgxlog.magic_client = default_client

In [ ]:
%%spanner
# back to using the default client
new uncle(str, str)
uncle("bob", "greg")
?uncle(X,Y)

# Processing the result of a query in python and using the result in a new query

In [ ]:
client = Client()
result = client.execute(f'''
    new friends(str, str, str)
    friends("bob", "greg", "clyde")
    friends("steven", "benny", "horace")
    friends("lenny", "homer", "toby")
    ?friends(X,Y,Z)
    ''')
print(result)
# Notice the default port is taken by the default client, hence the next port is used for the user's client


# now we'll showcase processing the result with native python...
result = result.split('\n')[3:-1]
result_tuples = []
for row in result:
    result_tuples.append(tuple(map(str.strip, row.split('|'))))
# future versions will do this internally

# lets filter our tuples with some predicate:
filtered = tuple(filter(lambda friends: 'bob' in friends or 'lenny' in friends, result_tuples))

# and feed the matching tuples into a new query:
client.execute('new buddies(str, str)')

for first, second, _ in filtered:
    client.execute(f'buddies("{first}", "{second}")')

result = client.execute("?buddies(First, Second)")
print(result)

# Developing and using your own IE functions

To make your own ie function you need to make a new python file and place it in rgxlog/user_ie_functions.
In the python file you need to create a new class that inherits from IEFunctionData.
By implementing its methods you are suppling the input format, output format, and the function itself.

In [ ]:
# example:

import spacy
sp = spacy.load('en_core_web_sm')

from rgxlog.engine.datatypes.primitive_types import DataTypes
from rgxlog.engine.ie_functions.ie_function_base import IEFunctionData

class Entities(IEFunctionData):
    def __init__(self):
        super().__init__()

    @staticmethod
    def ie_function(text):  # this is where you implement the actual processing
        entities = sp(text).ents
        return ((entity.text, spacy.explain(entity.label_)) for entity in entities)

    @staticmethod
    def get_input_types():  # this is where you define the inputs
        return [DataTypes.string]

    @staticmethod
    def get_output_types(output_arity):  # and this is where you define your outputs
        return tuple([DataTypes.string] * output_arity)

# place this file in rgxlog/user_ie_functions

next, we need to register the function by using the class name:

In [ ]:
import rgxlog
rgxlog.magic_client.register('Entities')

now we're free to use it:

In [ ]:
%%spanner
text =  "You've been living in a dream world, Neo.\
        As in Baudrillard's vision, your whole life has been spent inside the map, not the territory.\
        This is the world as it exists today.\
        Welcome to the desert of the real."
entities(Entity, Classification) <- Entities(text)->(Entity, Classification)
?entities(Entity, Classification)